In [2]:
from bs4 import BeautifulSoup
import requests
root_url = 'https://www.immoweb.be/en'
#cookie_url = root_url + '/cookie'
#cookies = requests.get(cookie_url).cookies
#print(cookies)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

### to get to the page-url for list of 10000  houses (only) and excluding life annuity sales (hence 2 filters applied)
### sorted by newest first, language set to English
pages = range(1) # assuming 333 pages if range is set to 333, assumption could be checked with last li class = "pagination__item" on the first page url (TBC)
listing_url_pages = []
listing_url_properties = []
for i in pages:
    page = i+1
    page_url=f"https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page={page}&orderBy=relevance&orderBy=newest"
    listing_url_pages.append(page_url)
print(listing_url_pages)
len(listing_url_pages)

['https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page=1&orderBy=relevance&orderBy=newest']


1

In [3]:
def list_propertie_urls(page_url):
    """function return a listing of property-urls 
    from an url to be provided as parameter (url representing of a listing-page of immoweb.be/en """
    list_1_page = []
    req_1_page = requests.get(page_url, headers=headers)
    page_soup = BeautifulSoup(req_1_page.text, 'html.parser')
    listings = page_soup.find_all('article', class_='card--result')
    for listing in listings:
        list_1_page.append(listing.find('a', class_='card__title-link', href=True)['href'])
    return (list_1_page)   

#list_propertie_urls('https://www.immoweb.be/en/search/house/for-sale?countries=BE&isALifeAnnuitySale=false&page=1&orderBy=relevance&orderBy=newestl)')
#list_propertie_url(listing_url_pages[-1]) # test when on last page

In [4]:
for i in listing_url_pages:
    listing_url_properties += (list_propertie_urls(i))
print(listing_url_properties)
len(listing_url_properties)

['https://www.immoweb.be/en/classified/house/for-sale/ath/7800/20234914', 'https://www.immoweb.be/en/classified/house/for-sale/lens/7870/20234898', 'https://www.immoweb.be/en/classified/house/for-sale/haine-saint-paul/7100/20234891', 'https://www.immoweb.be/en/classified/country-cottage/for-sale/modave/4577/20167723', 'https://www.immoweb.be/en/classified/villa/for-sale/grimbergen/1851/20234889', 'https://www.immoweb.be/en/classified/house/for-sale/lommel/3920/20234884', 'https://www.immoweb.be/en/classified/house/for-sale/dilbeek/1700/20234883', 'https://www.immoweb.be/en/classified/house/for-sale/arlon/6700/20234881', 'https://www.immoweb.be/en/classified/house/for-sale/ronse/9600/20234879', 'https://www.immoweb.be/en/classified/house/for-sale/anderlecht/1070/20234878', 'https://www.immoweb.be/en/classified/bungalow/for-sale/la-calamine/4720/20234877', 'https://www.immoweb.be/en/classified/house/for-sale/wiers/7608/20234866', 'https://www.immoweb.be/en/classified/house/for-sale/virto

30

In [28]:
# looping over a listing of properties and retrieve attributes for 1 property
listing_properties_detail = []
for i in listing_url_properties:
    print(i)
    req_1_house = requests.get(i, headers=headers)
    soup = BeautifulSoup(req_1_house.text, 'html.parser')
    listing_data = {}
    ### ID ###
    for x in soup.find('div', attrs={"class": "classified__header--immoweb-code"}):
       listing_data["Property_ID"] = x.text.split(":")[-1].strip()
    ### PRICE ###
    price_container = soup.find('p', attrs={"class": "classified__price"})
    sr_only_span = price_container.find('span', attrs={"class": "sr-only"})
    if sr_only_span:
      listing_data["Price"] = price_container.find('span', attrs={"class": "sr-only"}).text  
    ### MISC ###
    rows = soup.find_all('tr', attrs={"class" :'classified-table__row'})
    for row in rows:
    #print("row : ",row)
        if row.find('th', attrs={"class" :'classified-table__header'}):
            header = row.find('th', attrs={"class" :'classified-table__header'}).contents
        key_field =((header[0]).strip()) 
        #print(key_field)  
        if row.find('td', attrs={"class" :'classified-table__data'}):  
            data = row.find('td', attrs={"class" :'classified-table__data'}).contents
        value_field = ((data[0]).strip())
        #print(value_field) 
        relevant_keys = ["Living area","Terrace surface","Swimming pool", "Number of frontages", "Bedrooms", "How many fireplaces?", "Kitchen type", "Garden surface", "Swimming pool", "Type of building", "Furnished"]   
        if key_field in relevant_keys:
            listing_data[key_field]= value_field
    #####
    for key,value in listing_data.items():
        print(f"{key}: {value}")
      
    listing_properties_detail.append(listing_data)


https://www.immoweb.be/en/classified/house/for-sale/ath/7800/20234914
Property_ID: 20234914
Price: 407405€
Number of frontages: 4
Living area: 189
Bedrooms: 4
https://www.immoweb.be/en/classified/house/for-sale/lens/7870/20234898
Property_ID: 20234898
Price: 356405€
Number of frontages: 4
Living area: 189
Bedrooms: 4
https://www.immoweb.be/en/classified/house/for-sale/haine-saint-paul/7100/20234891
Property_ID: 20234891
Price: 120000€
Number of frontages: 2
Bedrooms: 2
https://www.immoweb.be/en/classified/country-cottage/for-sale/modave/4577/20167723
Property_ID: 20167723
Price: 390000€
Number of frontages: 4
Kitchen type: Installed
Bedrooms: 4
Garden surface: 1200
Terrace surface: 90
Type of building: House
https://www.immoweb.be/en/classified/villa/for-sale/grimbergen/1851/20234889
Property_ID: 20234889
Price: 999000€
Number of frontages: 4
Living area: 345
How many fireplaces?: 1
Kitchen type: Installed
Bedrooms: 4
Furnished: Yes
Terrace surface: 15
https://www.immoweb.be/en/classif

In [29]:
print(listing_properties_detail)



[{'Property_ID': '20234914', 'Price': '407405€', 'Number of frontages': '4', 'Living area': '189', 'Bedrooms': '4'}, {'Property_ID': '20234898', 'Price': '356405€', 'Number of frontages': '4', 'Living area': '189', 'Bedrooms': '4'}, {'Property_ID': '20234891', 'Price': '120000€', 'Number of frontages': '2', 'Bedrooms': '2'}, {'Property_ID': '20167723', 'Price': '390000€', 'Number of frontages': '4', 'Kitchen type': 'Installed', 'Bedrooms': '4', 'Garden surface': '1200', 'Terrace surface': '90', 'Type of building': 'House'}, {'Property_ID': '20234889', 'Price': '999000€', 'Number of frontages': '4', 'Living area': '345', 'How many fireplaces?': '1', 'Kitchen type': 'Installed', 'Bedrooms': '4', 'Furnished': 'Yes', 'Terrace surface': '15'}, {'Property_ID': '20234884', 'Price': '730000€', 'Number of frontages': '4', 'Living area': '419', 'Kitchen type': 'Hyper equipped', 'Bedrooms': '4', 'Furnished': 'No', 'Type of building': 'Villa'}, {'Property_ID': '20234883', 'Price': '219000€', 'Numb

In [30]:
import pandas as pd
df = pd.DataFrame.from_dict(listing_properties_detail)

In [33]:
df.shape



(30, 11)